In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [3]:
# Load API Credentials
with open('/Users/christinab/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

### Define Search

In [4]:
# set API call parameters 
location = 'New York, NY 10550'
term = 'Chinese'

In [5]:
# create the JSON data folder 
JSON_FILE = "Data/results_in_progress_NY_Chinese.json"
JSON_FILE

'Data/results_in_progress_NY_Chinese.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_NY_Chinese.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [10]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location='New York, NY 10019',
                                term='Chinese',
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
## How many results total?
total_results = results['total']
total_results


694

In [12]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

35

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [15]:
# set up a progress bar for your loop 
!pip install tqdm

In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/35 [00:00<?, ?it/s]

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location='Mount Vernon, NY 10002',
                                    term='Chinese', 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/35 [00:00<?, ?it/s]

In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,XsXLVWr1UZWVhKThNvNiaA,blue-willow-夜来湘-new-york-2,Blue Willow 夜来湘,https://s3-media2.fl.yelpcdn.com/bphoto/1E59vO...,False,https://www.yelp.com/biz/blue-willow-%E5%A4%9C...,668,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.5,"{'latitude': 40.76292, 'longitude': -73.976546}","[pickup, delivery]","{'address1': '40 W 56th St', 'address2': None,...",+12122132299,(212) 213-2299,1037.884778,NaN
1,bVkjBJlAIwKAj9Aw1KVWjA,dim-sum-palace-new-york-8,Dim Sum Palace,https://s3-media2.fl.yelpcdn.com/bphoto/k5mi8H...,False,https://www.yelp.com/biz/dim-sum-palace-new-yo...,1585,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.0,"{'latitude': 40.76015, 'longitude': -73.9893699}","[pickup, delivery]","{'address1': '334 W 46th St', 'address2': None...",+16468611910,(646) 861-1910,620.515163,$$
2,lQ8Fzgji9qmgRXkFcvAsVQ,chong-qing-xiao-mian-new-york-2,Chong Qing Xiao Mian,https://s3-media2.fl.yelpcdn.com/bphoto/9XObpV...,False,https://www.yelp.com/biz/chong-qing-xiao-mian-...,336,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 40.76513, 'longitude': -73.98753}","[pickup, delivery]","{'address1': '796 9th Ave', 'address2': '', 'a...",+12125828866,(212) 582-8866,94.299181,$$
3,EmJ7rfmv2y6lQZxgA8U__g,red-peony-chinese-cuisine-new-york-4,Red Peony Chinese Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/bD0jgT...,False,https://www.yelp.com/biz/red-peony-chinese-cui...,527,"[{'alias': 'shanghainese', 'title': 'Shanghain...",4.5,"{'latitude': 40.76278, 'longitude': -73.97589}","[delivery, pickup]","{'address1': '24 W 56th St', 'address2': None,...",+12123808883,(212) 380-8883,1092.523339,$$
4,Y92VtOkTZ6BirkJbUKCMOw,rong-cheng-house-new-york-4,Rong Cheng House,https://s3-media4.fl.yelpcdn.com/bphoto/vM4jaU...,False,https://www.yelp.com/biz/rong-cheng-house-new-...,119,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.0,"{'latitude': 40.76572, 'longitude': -73.98286}","[pickup, delivery]","{'address1': '242 W 56th St', 'address2': '', ...",+12122652226,(212) 265-2226,443.058683,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
422,_2eGAao4W3ww4vNB3RQiBA,salad-box-new-york-4,Salad Box,https://s3-media2.fl.yelpcdn.com/bphoto/r69gch...,False,https://www.yelp.com/biz/salad-box-new-york-4?...,79,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,"{'latitude': 40.71328, 'longitude': -74.00958}","[pickup, delivery]","{'address1': '44 Park Pl', 'address2': '', 'ad...",+16092932658,(609) 293-2658,1744.003157,NaN
423,TGa19wemPHqIwQCu08C0GQ,east-lake-express-new-york,East Lake Express,,False,https://www.yelp.com/biz/east-lake-express-new...,2,"[{'alias': 'chinese', 'title': 'Chinese'}]",1.5,"{'latitude': 40.70713, 'longitude': -74.00494}",[],"{'address1': '135 John St', 'address2': '', 'a...",+12127978939,(212) 797-8939,1751.951295,$
424,7IeZwAZI1o2orL8_Up3iJg,plump-dumpling-new-york-2,Plump Dumpling,,False,https://www.yelp.com/biz/plump-dumpling-new-yo...,2,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,"{'latitude': 40.7302739471197, 'longitude': -7...",[pickup],"{'address1': '299 E 11th St', 'address2': '', ...",+12122542868,(212) 254-2868,1412.775320,NaN
425,7Wj_qQpxGGUPSkwA_Jg2lQ,8a-sakura-sushi-new-york,8A Sakura Sushi,,False,https://www.yelp.com/biz/8a-sakura-sushi-new-y...,1,"[{'alias': 'chinese', 'title': 'Chinese'}]",1.0,"{'latitude': 40.713212415576, 'longitude': -74...","[pickup, delivery]","{'address1': '8 Murray St', 'address2': '', 'a...",+12123469788,(212) 346-9788,1621.809734,NaN
426,OPOmqhGguJY8W5LyenzQAQ,vivi-bubble-tea-new-york-10,ViVi Bubble Tea,https://s3-media3.fl.yelpcdn.com/bphoto/zHlvYT...,False,https://www.yelp.com/biz/vivi-bubble-tea-new-y...,151,"[{'alias': 'taiwanese', 'title': 'Taiwanese'},...",3.5,"{'latitude': 40.732567, 'longitude': -73.986229}","[pickup, delivery]","{'address1': '226 E 14th St', 'address2': None...",+12127778988,(212) 777-8988,1667.741121,$


In [19]:
# check for duplicates 
final_df.duplicated(subset='id').sum()

0

In [20]:
# drop duplicated ids and confirm there is no more 
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0